In [1]:
!pip install natsort

In [2]:
import pydicom
import os
from tqdm import tqdm
import numpy as np
import concurrent.futures
import json
from natsort import natsorted
from pydicom.encaps import encapsulate
from pydicom.uid import JPEG2000Lossless
import shutil
import random
from sklearn.model_selection import KFold
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [3]:
data_dir='/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'

In [4]:

# def process_serie(patient, serie_id, data_dir):
#     num_pix_row = []
#     num_pix_col = []
#     dicom_dir = f'{data_dir}/train_images/{patient}/{serie_id}'
#     problematic_serie_ids=[]
#     for dicom in os.listdir(dicom_dir): 
#         dicom_file = pydicom.dcmread(os.path.join(dicom_dir, dicom))
#         num_pix_col.append(dicom_file.pixel_array.shape[1])
#         num_pix_row.append(dicom_file.pixel_array.shape[0])


#     x_pixel = max(num_pix_row) if max(num_pix_row) == min(num_pix_row) else 640
#     y_pixel = max(num_pix_col) if max(num_pix_col) == min(num_pix_col) else 640
#     dicom_len = len(os.listdir(dicom_dir))

#     size = [x_pixel, y_pixel, dicom_len]
#     size_and_serie = [serie_id, size]
#     return size_and_serie

# def process_patient(patient, data_dir):
#     patient_dir = f'{data_dir}/train_images/{patient}'
#     results = []
#     for serie_id in os.listdir(patient_dir):
#         results.append(process_serie(patient, serie_id, data_dir))
#     return results

# def main(data_dir):
#     total_size = []
#     with concurrent.futures.ProcessPoolExecutor() as executor:
#         futures = [executor.submit(process_patient, patient, data_dir) for patient in os.listdir(f"{data_dir}/train_images")]
#         for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
#             total_size.extend(future.result())
#     return total_size
# data_dir='/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'
# total_size = main(data_dir)

In [5]:
with open('/kaggle/input/size-label/list.json', 'r') as f:
    total_size = json.load(f)

In [6]:
with open('/kaggle/input/label-coordinates/json_label_coordinates (2).json', 'r') as f:
    label_coordinates = json.load(f)

In [7]:
#####important: I should refine the way to impute missing data

def produce_label(serie_id):
    same_serie=0
    for i in range(len(label_coordinates)):
        if label_coordinates[i]['series_id']==serie_id:
            index_1=i-1
            index_2=i
            same_serie=same_serie+1
    for j in range(len((total_size))):
        if int(total_size[j][0])==serie_id:
            if same_serie==1:
                            # Extracting x, y, z values
                x_values_1 = [coord['x'] for coord in label_coordinates[index_2]['coordinates']]
                y_values_1 = [coord['y'] for coord in label_coordinates[index_2]['coordinates']]
                z_values_1 = [coord['z'] for coord in label_coordinates[index_2]['coordinates']]

                # Handling NaN values by imputing with the mean
                x_values_1 = [x / total_size[j][1][0] if not np.isnan(x) else np.nanmean(x_values_1) / total_size[j][1][0] for x in x_values_1]
                y_values_1 = [y / total_size[j][1][1] if not np.isnan(y) else np.nanmean(y_values_1) / total_size[j][1][1] for y in y_values_1]
                z_values_1 = [z / total_size[j][1][2] if not np.isnan(z) else np.nanmean(z_values_1) / total_size[j][1][2] for z in z_values_1]
                label_1= [val for triplet in zip(x_values_1, y_values_1, z_values_1) for val in triplet]
                label_2=[0]*15
                label=label_1+label_2
            elif same_serie==2:
                x_values_1 = [coord['x'] for coord in label_coordinates[index_1]['coordinates']]
                y_values_1 = [coord['y'] for coord in label_coordinates[index_1]['coordinates']]
                z_values_1 = [coord['z'] for coord in label_coordinates[index_1]['coordinates']]
                # Handling NaN values by imputing with the mean
                x_values_1 = [x / total_size[j][1][0] if not np.isnan(x) else np.nanmean(x_values_1) / total_size[j][1][0] for x in x_values_1]
                y_values_1 = [y / total_size[j][1][1] if not np.isnan(y) else np.nanmean(y_values_1) / total_size[j][1][1] for y in y_values_1]
                z_values_1 = [z / total_size[j][1][2] if not np.isnan(z) else np.nanmean(z_values_1) / total_size[j][1][2] for z in z_values_1]
                label_1= [val for triplet in zip(x_values_1, y_values_1, z_values_1) for val in triplet]
                x_values_2 = [coord['x'] for coord in label_coordinates[index_2]['coordinates']]
                y_values_2 = [coord['y'] for coord in label_coordinates[index_2]['coordinates']]
                z_values_2 = [coord['z'] for coord in label_coordinates[index_2]['coordinates']]

                # Handling NaN values by imputing with the mean
                x_values_2 = [x / total_size[j][1][0] if not np.isnan(x) else np.nanmean(x_values_2) / total_size[j][1][0] for x in x_values_2]
                y_values_2 = [y / total_size[j][1][1] if not np.isnan(y) else np.nanmean(y_values_2) / total_size[j][1][1]for y in y_values_2]
                z_values_2 = [z / total_size[j][1][2] if not np.isnan(z) else np.nanmean(z_values_2) / total_size[j][1][2] for z in z_values_2]
                label_2= [val for triplet in zip(x_values_2, y_values_2, z_values_2) for val in triplet]
                label=label_1+label_2
    return(label)

        

In [8]:
def standardize_dicom(patient,serie_id,data_dir):
    dicom_files = [f for f in os.listdir(f'{data_dir}/train_images/{patient}/{serie_id}')]
    dicom_files = natsorted(dicom_files)
    dicom_3d=[]
    for dicom in dicom_files:
        dir=f'{data_dir}/train_images/{patient}/{serie_id}/{dicom}'
        dicom_pix=pydicom.dcmread(dir).pixel_array
        dicom_pix=dicom_pix/dicom_pix.max()
        dicom_3d.append(dicom_pix)
    dicom_3d=torch.tensor(dicom_3d)
    return(dicom_3d)

In [9]:
dicom_3d=standardize_dicom(3343942613,4231470556,data_dir)
print(dicom_3d.unsqueeze(0).shape)

torch.Size([1, 22, 384, 384])


/tmp/ipykernel_18/2417833851.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  dicom_3d=torch.tensor(dicom_3d)


In [10]:
def resize(dicom_3d,new_shape=(25, 600, 600)):
    dicom_3d=dicom_3d.unsqueeze(0)
    dicom_3d=dicom_3d.unsqueeze(0)

    resized_tensor_trilinear = F.interpolate(dicom_3d, new_shape, mode='trilinear', align_corners=False)
    return(resized_tensor_trilinear)